In [1]:
cd ~/code/git/pudl/pudl

/Users/zaneselvans/code/git/pudl/pudl


In [2]:
import numpy as np
import pandas as pd
import pudl
import fercf1
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [3]:
#import eiaf923
%load_ext autoreload
%autoreload 1
%aimport pudl
%aimport fercf1
#%aimport eiaf923

In [4]:
engine = create_engine('postgresql://catalyst@localhost:5432/pudl_sandbox')
pudl.Base.metadata.drop_all(engine)
pudl.Base.metadata.create_all(engine)

In [5]:
Session = sessionmaker(bind=engine)
session = Session()

In [6]:
pudl.Base.metadata.drop_all(engine)
pudl.init_db(pudl.Base)

In [11]:
map_eia923_ferc1_file = '../results/id_mapping/mapping_eia923_ferc1.xlsx'

util_map  = pd.read_excel(map_eia923_ferc1_file,'utilities_output',
                              converters={'utility_id':int,
                                          'utility_name':str,
                                          'respondent_id_ferc1':int,
                                          'respondent_name_ferc1':str,
                                          'operator_id_eia923':int,
                                          'operator_name_eia923':str})

utils_eia923 = util_map[['operator_id_eia923','operator_name_eia923','utility_id']]
utils_eia923 = utils_eia923.drop_duplicates('operator_id_eia923')
utils_ferc1 = util_map[['respondent_id_ferc1','respondent_name_ferc1','utility_id']]
utils_ferc1 = utils_ferc1.drop_duplicates('respondent_id_ferc1')

plant_map = pd.read_excel(map_eia923_ferc1_file,'plants_output',
                              converters={'plant_id':int,
                                          'plant_name':str,
                                          'respondent_id_ferc1':int,
                                          'respondent_name_ferc1':str,
                                          'plant_name_ferc1':str,
                                          'plant_id_eia923':int,
                                          'plant_name_eia923':str,
                                          'operator_name_eia923':str,
                                          'operator_id_eia923':int})

plants_eia923 = plant_map[['plant_id_eia923','plant_name_eia923','plant_id']]
plants_eia923 = plants_eia923.drop_duplicates('plant_id_eia923')
plants_ferc1 = plant_map[['plant_name_ferc1','respondent_id_ferc1','plant_id']]
plants_ferc1 = plants_ferc1.drop_duplicates(['plant_name_ferc1','respondent_id_ferc1'])

In [53]:
for df in [utils_eia923, utils_ferc1, plants_eia923, plants_ferc1]:
    print(df.shape[0]-df.dropna().shape[0])

1
1
3
2


In [12]:
plants_ferc1[plants_ferc1.isnull().any(axis=1)]

,plant_name_ferc1,respondent_id_ferc1,plant_id
578,NaN,428,420
1432,NaN,NaN,1124


In [ ]:
def demo_func(tabname):
    tabmap = {"boiler_fuel":0,
              "generators":1,
              "fuel_delivered":2 }
    df = pd.read_excel("myfile.xlsx",sheetnum=tabmap[tabname])